<a href="https://colab.research.google.com/github/jad-r-s/Jad_DTSC3020_Fall2025/blob/main/Assignment5_Ch_10%2611.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment5: CRM Cleanup @ **DalaShop**
*Files (Ch.10), Exceptions (Ch.10), Unit Tests (Ch.11), and Regular Expressions*  
.....

**Total: 3 points**  (Two questions, 1.5 pts each)  

> This assignment is scenario-based and aligned with Python Crash Course Ch.10 (files & exceptions), Ch.11 (unit testing with `unittest`), and Regular Expressions.

## Scenario
You are a data intern at an online retailer called **DalaShop**.  
Sales exported a **raw contacts** file from the CRM. It contains customer names, emails, and phone numbers, but the formatting is messy and some emails are invalid.  
Your tasks:

1. **Clean** the contacts (Files + Exceptions + Regex).  
2. **Write unit tests** to make sure your helper functions work correctly and keep working in the future.

## Data file (given by the company): `contacts_raw.txt`
Use this exact sample data (you may extend it for your own testing, but do **not** change it when submitting).  
Run the next cell once to create the file beside your notebook.

In [4]:
# Create the provided company dataset file
with open("contacts_raw.txt", "w", encoding="utf-8") as f:
    f.write('Alice Johnson <alice@example.com> , +1 (469) 555-1234\nBob Roberts <bob[at]example.com> , 972-555-777\nSara M. , sara@mail.co , 214 555 8888\n"Mehdi A." <mehdi.ay@example.org> , (469)555-9999\nDelaram <delaram@example.io>, +1-972-777-2121\nNima <NIMA@example.io> , 972.777.2121\nduplicate <Alice@Example.com> , 469 555 1234')
print("Wrote contacts_raw.txt with sample DalaShop data.")

Wrote contacts_raw.txt with sample DalaShop data.


## Q1 (1.5 pts) — CRM cleanup with Files, Exceptions, and Regex
Implement `q1_crm_cleanup.py` to:

1. **Read** `contacts_raw.txt` using `pathlib` and `with`. If the file is missing, **handle** it gracefully with `try/except FileNotFoundError` (print a friendly message; do not crash).
2. **Validate emails** with a simple regex (`r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"`).  
   - Trim whitespace with `strip()` before checking.  
   - Use **full** matching (not partial).
3. **Normalize phone numbers:** remove all non-digits (e.g., with `re.sub(r"\D", "", raw)`).  
   - If the result has **≥ 10 digits**, keep the **last 10 digits**.  
   - Otherwise, return an **empty string** (`""`).
4. **Filter rows:** keep **only** rows with a valid email.
5. **Deduplicate:** remove duplicates by **email** using **case-insensitive** comparison (e.g., `email.casefold()`). **Keep the first occurrence** and drop later duplicates.
6. **Output CSV:** write to `contacts_clean.csv` with **columns exactly** `name,email,phone` (UTF-8).  
7. **Preserve input order:** the order of rows in `contacts_clean.csv` must match the **first appearance** order from the input file. **Do not sort** the rows.

**Grading rubric (1.5 pts):**
- (0.4) File read/write via `pathlib` + graceful `FileNotFoundError` handling  
- (0.5) Correct email regex validation + filtering  
- (0.4) Phone normalization + case-insensitive de-dup (keep first)  
- (0.2) Clean code, clear names, minimal docstrings/comments

In [7]:
# Write your answer here
"""
q1_crm_cleanup.py
Cleans raw CRM contact data for DalaShop
"""

from pathlib import Path
import re
import csv

# regex for valid email addresses
EMAIL_REGEX = re.compile(r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$")

def is_valid_email(email: str) -> bool:
    """Return true if email is valid using full regex match"""
    return bool(EMAIL_REGEX.fullmatch(email.strip()))

def normalize_phone(raw_phone: str) -> str:
    """Remove non-digits and keep the last 10 digits if available"""
    digits = re.sub(r"\D", "", raw_phone)
    return digits[-10:] if len(digits) >= 10 else ""

def main():
    input_path = Path("contacts_raw.txt")
    output_path = Path("contacts_clean.csv")

    try:
        raw_text = input_path.read_text(encoding="utf-8")
    except FileNotFoundError:
        print("Error: contacts_raw.txt not found. Please make sure the file exists.")
        return

    seen_emails = set()
    cleaned_rows = []

    for line in raw_text.strip().splitlines():
        parts = [p.strip() for p in re.split(r"[<>,]", line) if p.strip()]
        # attempt to extract name, email, and phone
        if len(parts) >= 3:
            name, email, phone = parts[0], parts[1], parts[2]
        elif len(parts) == 2:
            # handle missing name or phone
            if "@" in parts[0]:
                email, phone = parts[0], parts[1]
                name = ""
            else:
                name, email = parts[0], parts[1]
                phone = ""
        else:
            continue  # skip unrecognized rows

        if not is_valid_email(email):
            continue

        normalized_phone = normalize_phone(phone)
        email_key = email.casefold()

        if email_key not in seen_emails:
            seen_emails.add(email_key)
            cleaned_rows.append({
                "name": name.strip('"').strip(),
                "email": email.strip(),
                "phone": normalized_phone
            })

    # write to contacts_clean.csv
    with output_path.open("w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["name", "email", "phone"])
        writer.writeheader()
        writer.writerows(cleaned_rows)

    print(f"Cleaned contacts written to {output_path.name}")

if __name__ == "__main__":
    main()

Cleaned contacts written to contacts_clean.csv


## Q2 (1.5 pts) — Unit testing with `unittest`
Create tests in `test_crm_cleanup.py` that cover at least:

1. **Email validation**: valid/invalid variations.  
2. **Phone normalization**: parentheses, dashes, spaces, country code; too-short cases.  
3. **Parsing**: from a small multi-line string (not from a file), assert the exact structured rows (name/email/phone).  
4. **De-duplication**: demonstrate that a case-variant duplicate email is dropped (first occurrence kept).




In [6]:
# Write your answer here
"""
Unit tests for q1_crm_cleanup.py.
Covers email validation, phone normalization, parsing, and deduplication.
"""

import unittest
from q1_crm_cleanup import is_valid_email, normalize_phone
import re
from q1_crm_cleanup import EMAIL_REGEX


class TestCRMHelpers(unittest.TestCase):
    def test_valid_emails(self):
        valid = [
            "alice@example.com",
            "bob.smith123@domain.co.uk",
            "MEHDI-AY@EXAMPLE.ORG",
            "x_y.z+test@mail.io",
        ]
        for email in valid:
            self.assertTrue(is_valid_email(email))

    def test_invalid_emails(self):
        invalid = [
            "bob[at]example.com",
            "sara@mail",
            "noatsymbol.com",
            "@missingname.com",
            "john@.com",
        ]
        for email in invalid:
            self.assertFalse(is_valid_email(email))

    def test_phone_normalization(self):
        cases = {
            "+1 (469) 555-1234": "4695551234",
            "972-555-777": "",
            "(214) 555 8888": "2145558888",
            "+1-972-777-2121": "9727772121",
            "972.777.2121": "9727772121",
            "1234567890123": "2345678901",
        }
        for raw, expected in cases.items():
            self.assertEqual(normalize_phone(raw), expected)


class TestParsingAndDedup(unittest.TestCase):
    def parse_lines(self, text):
        seen = set()
        rows = []
        for line in text.strip().splitlines():
            parts = [p.strip() for p in re.split(r"[<>,]", line) if p.strip()]
            if len(parts) >= 3:
                name, email, phone = parts[0], parts[1], parts[2]
            elif len(parts) == 2:
                if "@" in parts[0]:
                    email, phone = parts[0], parts[1]
                    name = ""
                else:
                    name, email = parts[0], parts[1]
                    phone = ""
            else:
                continue

            if not EMAIL_REGEX.fullmatch(email.strip()):
                continue

            digits = re.sub(r"\D", "", phone)
            normalized_phone = digits[-10:] if len(digits) >= 10 else ""
            email_key = email.casefold()

            if email_key not in seen:
                seen.add(email_key)
                rows.append((name.strip('"').strip(), email.strip(), normalized_phone))
        return rows

    def test_parsing_and_deduplication(self):
        raw = """Alice Johnson <alice@example.com> , +1 (469) 555-1234
        duplicate <Alice@Example.com> , 469 555 1234
        Mehdi <mehdi@example.org> , 972.888.9999
        invalid <bob[at]example.com> , 9725550000
        """

        rows = self.parse_lines(raw)
        expected = [
            ("Alice Johnson", "alice@example.com", "4695551234"),
            ("Mehdi", "mehdi@example.org", "9728889999")
        ]
        self.assertEqual(rows, expected)


if __name__ == "__main__":
    unittest.main()

ModuleNotFoundError: No module named 'q1_crm_cleanup'

## Grading rubric (total 3 pts)
- **Q1 (1.5 pts)**  
  - (0.4) File I/O with `pathlib` + graceful `FileNotFoundError` handling  
  - (0.5) Email validation (regex + strip + full match) and filtering  
  - (0.4) Phone normalization and **case-insensitive** de-duplication (keep first)  
  - (0.2) Code clarity (names, minimal docstrings/comments)
- **Q2 (1.5 pts)**  
  - (0.6) Meaningful coverage for email/phone functions (valid & invalid)  
  - (0.6) Parsing & de-dup tests that assert exact expected rows  
  - (0.3) Standard `unittest` structure and readable test names
